# PSB sequence

This includes the correct ramp rate and SET feedback as well.

**TODO**
- Run X steps of detuning before applying feedback
- Only apply feedback after each change in J
- Add buffer times to make sure that everything is syncing correctly

In [1]:
%matplotlib inline
import local_broom

Added 'quench/' and 'quench/libraries' for easy importing.


In [373]:
from zhinst.toolkit import Session, CommandTable, Sequence, Waveforms, SHFQAChannelMode

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from qcodes import Instrument
import time
import warnings
from monty import Monty
import MDAC
from may.dots import getvoltages
from may.custom_devices import connect_to_gb, newSiDot
from liveplot import LivePlot
from fridge import Fridge

from qcodes.instrument_drivers.oxford.MercuryiPS_VISA import MercuryiPS


## Setup

### Common Functions

In [374]:
def timeToSamples(time, samplingRateDivider):
    # returns the number of samples divisible by 16 given a time (in seconds) and sampling rate divider
    samples_raw = time * (1/(2**samplingRateDivider))/0.5e-9
    samples_modulo = int(samples_raw) % 16
    samples = int(samples_raw) - int(samples_modulo)
    return samples

# custom curve fit
#amps = [1, 0.85, 0.75, 0.65,  0.5, 0.35, 0.25, 0.2, 0.125, 0.05]
#voltages = [0.34, 0.268, 0.207, 0.158, 0.094, 0.0481,  0.026, 0.0183, 0.01, 0.0057]

#alt_fit = np.poly1d(np.polyfit(amps, voltages, 2))
# Hard coding alt_fit values in case something goes horribly wrong

def voltToDbm(volt, dbmrange):
    # Ok yes this can be better, deal with it
    if dbmrange != 0:
        raise Exception("This function only works with a dBm range of 0.")
    
    # if volt > 0.34 or volt < -0.34:
    #     raise ValueError(f"Given voltage ({volt} V) is greater than max output of SHFQC (0.34 V)")
    
    if volt < 0:
        amplitude = 1/300*(np.sqrt(3e5*-volt + 529) - 23)
        return -amplitude
    else:
        amplitude = 1/300*(np.sqrt(3e5*volt + 529) - 23)
        return amplitude


def autodb(res):
    """Change a.u. to dB power"""
    return 10*np.log10(np.abs(res)**2/50*1000)


def get_results(result_node, timeout):
    wave_data_captured = {}
    wave_data_captured[result_node] = False
    start_time = time.time()
    captured_data = {}
    while not all(wave_data_captured.values()):
        if start_time + timeout < time.time():
            print(captured_data)
            raise TimeoutError('Timeout before all samples collected.')
        test = session.poll()
        for node, value in test.items():
            node = session.raw_path_to_node(node)
            for v in value:
                if node not in captured_data:
                    captured_data[node] = [v['vector']]
                else:
                    captured_data[node].append(v['vector'])
            if len(captured_data[node]) >= 1:  # readout 1 point
                wave_data_captured[node] = True
                # total_num_data = sum([len(element) for element in captured_data[node]])
    data = captured_data[result_node][0]
    return data

def cmdtable(ct, amplitude, length, wave_index, ct_index):
    """
    Load a default command table with a sin/cos wave (used throughout the documentation)
    """
    ct.table[ct_index].waveform.index = wave_index
    ct.table[ct_index].amplitude00.value = amplitude  # all in dBm
    ct.table[ct_index].amplitude01.value = -amplitude
    ct.table[ct_index].amplitude10.value = amplitude
    ct.table[ct_index].amplitude11.value = amplitude
    ct.table[ct_index].waveform.length = length  # in samples
    ct.table[ct_index].waveform.samplingRateDivider = samplingDivider  # inherit global

### Connect

In [376]:
# connect to instruments

DEVICE_ID = 'DEV12158'
SERVER_HOST = 'localhost'

session = Session(SERVER_HOST)  # connect to data server
device = session.connect_device(DEVICE_ID)  # connect to device


try:
    mdac = Instrument.find_instrument("mdac")
    mdac.close()
except KeyError:
    print('Attempting to remove instrument with name mdac. Does not exist')

mdac = MDAC.MDAC('mdac', 'ASRL11::INSTR')
gb_control_si = connect_to_gb(mdac)  # Create our custom MDAC mappings
si = newSiDot(mdac)
fridge = Fridge("BlueFors_LD")
#magnet = MercuryiPS('magnet15', 'TCPIP0::192.168.0.106::7020::SOCKET')

Connected to: QNL Sydney MDAC (serial:0x32353532353551040030003E, firmware:1.8) in 4.21s
Loading GB driver and connecting
Manually Open Breakout_box_top_ch15_LCB
Manually Open Breakout_box_bot_ch09_RCB
Manually Open Breakout_box_bot_ch17_RG
Manually Open Breakout_box_top_ch17_ResB
Manually Open Breakout_box_top_ch19_P3
Manually Open Breakout_box_top_ch13_P2
Manually Open Breakout_box_top_ch04_P1
Manually Open Breakout_box_top_ch02_SETB
Manually Open Breakout_box_bot_ch04_SRB
Manually Open Breakout_box_bot_ch06_SLB
Manually Open Breakout_box_top_ch10_ST
Manually Open Breakout_box_top_ch25_bias
Using base URL https://qphys1114.research.ext.sydney.edu.au/therm_flask/BlueFors_LD


In [377]:
if False:
    session.disconnect_device(DEVICE_ID)
    try:
        mdac = Instrument.find_instrument("mdac")
        mdac.close()
    except KeyError:
        print('Attempting to remove instrument with name mdac. Does not exist')

In [378]:
if False:
    magnet._set_target_and_ramp('z', 'simul', 0.723)
    time.sleep(700) #ramping at approx 1mT/s

In [379]:
#session.disconnect_device(DEVICE_ID)

In [380]:
experiment = {
    "desc": "PSB sequence."
}

monty = Monty("rf.detuning", experiment)

Loading existing experiment (ignoring given experiment parameters)
Note that no experimental data has been loaded.
Next run will have id 306


### Define variables

In [381]:

# lengths and averages
seq_averages = 200  # seqc averages

NUM_DETUNING = 100  # 300
NUM_J = 1

# readout signals
readout_gain = 0.95  # If we set this to 1, then output overloads
readout_freq = 406.6e6  # (Hz), try 406.2

# drive line amplitudes (maximum 340mV, give in terms of V)
# pulse parameters
MIXED_AMP = {  # voltage amplitude for mixed state preperation
    "P1": 0.0,
    "P2": -0.0,
}
MES_AMP_START = {  # voltages for stepping over when sweeping
    "P1": 0.0,
    "P2": -0.0,
    "J": 0,
}
MES_AMP_END = {
    "P1": -0.1,
    "P2": 0.1,
    "J": 0.0,
}

# timings in seconds
init_len = 600e-6
read_len = 100e-6
wait_and_settle = 3e-3
trigger_time = 8e-3  # internal trigger holdoff time (metronome)
buffer = 10e-6

# AC decay ramp rate
ramp_rate = 13e-5  # V/sample

# powers
input_pwr = 5
output_pwr = -10
dr_pwr = 0

# gate sampling time
samplingDivider = 6

# feedback parameters
FB_TARGET = -24.1  # give in terms of power output
FB_STEPSIZE = 8e-1
FB_SLOPE = -1  # 1 or -1 depending on the slope to lock onto

# internal
read_samples = timeToSamples(read_len, samplingDivider)
steps = np.logspace(-4.5, -2.5, NUM_J)

# DC voltages
si.P1(1.76)
si.P2(1.96)
si.ST(3.655)

In [382]:
# adjust variables
curr_st_amp = 0.0  # NEED TO FIX THIS...

# Create channel maps for simplicity

chan = {
    "measure": device.qachannels[0],  # measure and acquire lines
    "ST": device.sgchannels[0],
    "P1": device.sgchannels[1],  # drive P1 line
    "P2": device.sgchannels[2],  # drive P2 line
    "J": device.sgchannels[3],  # drive J line
}

drive_chans = ["ST", "P1", "P2", "J"]  # match keys above

# current command tables
cts = {c: CommandTable(chan[c].awg.commandtable.load_validation_schema()) for c in drive_chans}

j_steps = np.linspace(MES_AMP_START["J"], MES_AMP_END["J"], NUM_J)
p1_steps = np.linspace(MES_AMP_START["P1"], MES_AMP_END["P1"], NUM_DETUNING)
p2_steps = np.linspace(MES_AMP_START["P2"], MES_AMP_END["P2"], NUM_DETUNING)

In [383]:
# save all the parameters about this experiment that we will want to know
params = {
    "amplitude_volts": {
        "mixed_pulse": MIXED_AMP,
        "measure_pulse_start": MES_AMP_START,
        "measure_pulse_end": MES_AMP_END,
        "p_ramp_rate": ramp_rate,
    },
    "readout": {
        "freq": readout_freq,
        "gain": readout_gain,
        "time": read_len
    },
    "timings_sec": {
        "mixed initilise": init_len,
        "read": read_len,
        "settle": wait_and_settle,
        "trigger": trigger_time,
        "buffer": buffer,
        "sampling_divider": samplingDivider,
    },
    "feedback": {
        "target": FB_TARGET,
        "stepsize": FB_STEPSIZE,
        "slope": FB_SLOPE
    },
    "powers": {
        "acq_in": input_pwr,
        "acq_out": output_pwr,
        "drive": dr_pwr,
    },
    "averaging": {
        "seqc_averages": seq_averages,
        "num_detuning": NUM_DETUNING,
        "num_j": NUM_J,
    },
    "gates": getvoltages(mdac),
    "temp": f"Mixing chamber {fridge.temp()} K",
}

In [384]:
# check we can fit all the points into memory
MAX_MEASUREMENTS = 2**19
if seq_averages > MAX_MEASUREMENTS:
    raise OverflowError("Requested too many points to be measured. (Use around 500,000 points)")

### Configure Channels

In [385]:
with device.set_transaction():
    # setup drive channels
    for c in drive_chans:
        chan[c].output.range(dr_pwr)  # in dBm
        chan[c].output.rflfpath(0)  # use LF not RF (1 for RF)
    
        # set the center synth frequency (oscillator frequency)
        synth = chan[c].synthesizer()
        device.synthesizers[synth].centerfreq(0)  # in Hz
        chan[c].output.on(1)  # enable output
    
        chan[c].awg.outputamplitude(1.0)  # overall amplitude scaling factor (don't really need to change)
        chan[c].oscs[0].freq(0)  # oscillator 1 frequency (Hz) disable for DC
        chan[c].oscs[1].freq(0)  # oscillator 2 frequency (Hz)
        chan[c].awg.modulation.enable(1)  # start digital modulation
    
        chan[c].marker.source(0)  # setup the AWG trigger 1 (is this an input trigger option? doesn't seem necessary)
        # see manual page p235 for all trigger options
        chan[c].awg.auxtriggers[0].channel(8)  # 8=use internal trigger, 1024=use software trigger

    # setup measure channel
    
    chan["measure"].output.rflfpath(0)  # use LF mode not RF (for signals under 600Mhz)
    chan["measure"].input.rflfpath(0)
    chan["measure"].oscs[0].freq(readout_freq)  # CW frequency (in LF mode)
    chan["measure"].oscs[0].gain(readout_gain)  # If we set this to 1, then output overloads

    # configure these based on how the sweeper works internally
    # See https://docs.zhinst.com/zhinst-utils/en/latest/_modules/zhinst/utils/shf_sweeper.html#ShfSweeper
    chan["measure"].spectroscopy.delay(0)  # integration delay in units of second
    chan["measure"].spectroscopy.length(100e-6*2e9)#timeToSamples(read_len, 0))  # integration time length in units of number of samples (usually integration_time*sampling_rate)
    # setup when the spectroscopy is triggered
    chan["measure"].spectroscopy.trigger.channel("chan0seqtrig0")  # make sure to use the trigger coming from the sequencer code
    # setup result parameters
    chan["measure"].spectroscopy.result.averages(seq_averages)  # number of averages (always average in software not hardware)
    chan["measure"].spectroscopy.result.length(2)  # number of results
    chan["measure"].spectroscopy.result.enable(0)  # disable result logger
    chan["measure"].spectroscopy.result.mode('cyclic')  # sequential readout for averaging
    chan["measure"].spectroscopy.envelope.enable(0)  # changes into continuous mode

    chan["measure"].configure_channel(
        center_frequency=0,  # in units of Hz  # minimum of 600MHz for RF mode
        input_range=input_pwr,  # in units of dBm
        output_range=output_pwr,  # in units of dBm
        mode=SHFQAChannelMode.SPECTROSCOPY,  # SHFQAChannelMode.READOUT or SHFQAChannelMode.SPECTROSCOPY
    )
    
    chan["measure"].input.on(1)
    chan["measure"].output.on(1)

    chan["measure"].generator.auxtriggers[1].channel("inttrig")  # i believe this is overwritten by the following line
    chan["measure"].generator.configure_sequencer_triggering(
        aux_trigger=8,  # alternatively use 8=internal trigger, or "software_trigger0" to use the software triggering system
        play_pulse_delay=0
    )

    device.system.internaltrigger.repetitions(seq_averages)  # make sure that this matches how many pulses we are sending
    device.system.internaltrigger.holdoff(trigger_time)  # init_len + 2*wait_and_settle + 100e-6*2)  # how long to wait between retriggers (increments of 100ns)


### Define Sequences

*P1/P2 sequence explanation*: The sequence starts at a "home" point with 0 V on the P1/P2/J pulsing lines. After waiting for a trigger, the sequence will immediately go to it's initialisation point (executeTableEntry(0)) and wait for a set period of time (here: wait_and_settle, read_len). After a J pulse between wait_and_settle and read_len events P1 and P2 will be pulsed to an arbitrary point (executeTableEntry(1)) which is uploaded and updated every time in python after SET feedback has occured.

In [386]:
seqc_program_p1 = f"""
// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat({seq_averages}) {{

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init.
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
    playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});           // read reference

    executeTableEntry(1);                                                               // read
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
}}
"""

seqc_program_p2 = f"""
// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat({seq_averages}) {{

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init.
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
    playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});           // read reference

    executeTableEntry(1);                                                               // read
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
}}
"""

seqc_program_j = f"""
// Assign a single channel waveform to wave table entry 0
wave w_j = ones({timeToSamples(init_len, samplingDivider)});
assignWaveIndex(1,2, w_j, 0);

// Reset the oscillator phase
resetOscPhase();

repeat({seq_averages}) {{

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    playZero({timeToSamples(init_len, samplingDivider)},  {samplingDivider});           // mixed state init.
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle

    executeTableEntry(1);                                                               // read reference
    
    // playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});           // read
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
    playZero(32);    // wait and settle
}}
"""



seqc_program_st = f"""
// Assign a single channel waveform to wave table entry 0
wave w_st = ones({timeToSamples(wait_and_settle, 9)});
assignWaveIndex(1,2, w_st, 0);

repeat({seq_averages}) {{
    waitDigTrigger(1);
    
    playZero({timeToSamples(init_len, samplingDivider)},  {samplingDivider});
    playZero({timeToSamples(wait_and_settle-10*buffer, samplingDivider)},  {samplingDivider});
    //playZero({timeToSamples(buffer, samplingDivider)},  {samplingDivider});
    
    executeTableEntry(0);
}}
"""

In [387]:
seqc_program_prior_read = f"""
repeat({seq_averages}) {{
    waitDigTrigger(1);
    
    setTrigger(1);
    setTrigger(0);
}}
"""

readout_prog_code = f"""
setTrigger(0); // Set low as this starts the spectroscopy readout....

repeat({seq_averages}) {{
    waitDigTrigger(1);
    
    playZero(224); // lineup with SG trigger (224 samples = lines up with SG trigger);
    
    playZero({timeToSamples(init_len, samplingDivider)},  {samplingDivider});
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});
    playZero({timeToSamples(buffer, samplingDivider)},  {samplingDivider});

    setTrigger(1);  // trigger the output. As this matches "chan0seqtrig0" the spectroscopy is started
    setTrigger(0);

    playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});
    playZero({timeToSamples(buffer, samplingDivider)},  {samplingDivider});

    setTrigger(1);  // trigger the output. As this matches "chan0seqtrig0" the spectroscopy is started
    setTrigger(0);

    playZero({timeToSamples(buffer, samplingDivider)},  {samplingDivider});
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});
    }}

"""

In [388]:
# create waveforms
default_amp = 0.01

# Create mixed state
samples = timeToSamples(init_len, samplingDivider)
mixed_p = np.linspace(voltToDbm(0.34 - 0.34*samples*ramp_rate, 0), voltToDbm(0.34, 0), samples)

# Read pulse driver
samples = timeToSamples(read_len, samplingDivider)
# read_p = np.linspace(voltToDbm(default_amp, dr_pwr),
#                        voltToDbm(default_amp+default_amp*samples*ramp_rate, dr_pwr), samples)
read_p = np.linspace(voltToDbm(0.34 - 0.34*samples*ramp_rate, 0), voltToDbm(0.34, 0), samples)
# read_p = np.linspace(voltToDbm(0.34, 0), voltToDbm(0.34+ 0.34*samples*ramp_rate, 0), samples)

del samples

In [389]:
# upload drive code

# P1
seq = Sequence()
seq.code = seqc_program_p1
seq.waveforms = Waveforms()
seq.waveforms[0] = mixed_p
seq.waveforms[1] = read_p
chan["P1"].awg.load_sequencer_program(seq)
chan["P1"].awg.write_to_waveform_memory(seq.waveforms)
print(f"_________ {chan["P1"]} _________")
print(seq.code)

# P2
seq = Sequence()
seq.code = seqc_program_p2
seq.waveforms = Waveforms()
seq.waveforms[0] = mixed_p
seq.waveforms[1] = read_p
chan["P2"].awg.load_sequencer_program(seq)
chan["P2"].awg.write_to_waveform_memory(seq.waveforms)
print(f"_________ {chan["P2"]} _________")
print(seq.code)

# J
chan["J"].awg.load_sequencer_program(seqc_program_j)
print(f"_________ {chan["J"]} _________")
print(seqc_program_j)

# ST
chan["ST"].awg.load_sequencer_program(seqc_program_st)
print(f"_________ {chan["ST"]} _________")
print(seqc_program_st)

# upload wavevforms with sequence code as the memory must be allocated first

# upload measure code
#chan["measure"].generator.load_sequencer_program(readout_prog_code)
#chan["measure"].generator.load_sequencer_program(seqc_program_prior_read)

_________ /dev12158/sgchannels/1 _________

// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat(200) {

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init.
    
    playZero(93744,  6);    // wait and settle
    playZero(3120,  6);           // read reference

    executeTableEntry(1);                                                               // read
    
    playZero(93744,  6);    // wait and settle
}

_________ /dev12158/sgchannels/2 _________

// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat(200) {

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init.


### Setup Command tables

In [390]:
# load mixed state command channels
for c in ["P1", "P2"]:
    cmdtable(cts[c],
             amplitude= voltToDbm(MIXED_AMP[c], chan[c].output.range()),
             length=timeToSamples(init_len, samplingDivider),
             wave_index=0,
             ct_index=0,
            )

## Run

### Run functions

In [391]:
def uploadcts():
    """Upload the command tables to the device"""
    for c in drive_chans:
        chan[c].awg.commandtable.upload_to_device(cts[c])

In [392]:
def synchchannels(chans):
    """Sync the given channels, unsyncing the rest. Assumes that the internal trigger and measure channel should always be synced."""
    for i in range(6):
        device.sgchannels[i].synchronization.enable(0)
    time.sleep(1)
    for c in drive_chans:
        if c in chans:  # sync
            chan[c].synchronization.enable(1)
    chan["measure"].synchronization.enable(1)
    device.system.internaltrigger.synchronization.enable(1)

In [393]:
def waitForInternalTrigger(progress=True, leave=True):
    """
    Waits for the internal trigger to finish running and shows the current progress.
    Progress = if show tqdm progress meter
    Leave = if keep progress bar afterwards
    """
    if progress:
        pbar = tqdm(total=100, leave=leave, desc="Internal trigger")
    while device.system.internaltrigger.progress() != 1.0:
        p = int(device.system.internaltrigger.progress()*100)
        if progress:
            pbar.update(p-pbar.n)
        time.sleep(0.001)
    if progress:
        pbar.update(100-pbar.n)
        pbar.close()

In [394]:
def calculatefeedback(data):
    """Calculate the appropriate feedback adjustment to ST."""
    # use last datapoint
    global curr_st_amp
    error = FB_TARGET - 10*np.log10(np.abs(data[-1])**2/50*1000)
    curr_st_amp += error*FB_STEPSIZE*FB_SLOPE

    # update command table
    cmdtable(cts["ST"],
             amplitude= voltToDbm(curr_st_amp, chan["ST"].output.range()),
             length=timeToSamples(trigger_time-2e-3, 9),
             wave_index=0,
             ct_index=0,
            )
    return curr_st_amp

def calculatefeedbackalt(data):
    """Calculate the appropriate feedback adjustment to ST."""
    # use last datapoint
    unit_amp = 0.34
    error = FB_TARGET - 10*np.log10(np.abs(data[-1])**2/50*1000)
    st_err = error*FB_STEPSIZE*FB_SLOPE
    st_err = -1 if st_err < -1 else (1 if st_err > 1 else st_err)
    st_corr = st_err*unit_amp

    # update command table
    cmdtable(cts["ST"],
             amplitude= voltToDbm(st_corr, chan["ST"].output.range()),
             length=timeToSamples(trigger_time-2e-3, 9),
             wave_index=0,
             ct_index=0,
            )
    return st_corr

In [395]:
def movemeasurement(p1, p2, j):
    """Move P1/P2/J to measure the next appropriate datapoint."""
    cmdtable(cts["P1"],
             amplitude=voltToDbm(p1, chan["P1"].output.range()),
             length=timeToSamples(buffer + read_len + buffer, samplingDivider),
             wave_index=1,
             ct_index=1,
            )
    cmdtable(cts["P2"],
             amplitude=voltToDbm(p2, chan["P2"].output.range()),
             length=timeToSamples(buffer + read_len + buffer, samplingDivider),
             wave_index=1,
             ct_index=1,
            )
    cmdtable(cts["J"],
             amplitude= voltToDbm(j, chan["J"].output.range()),
             length=timeToSamples(buffer + read_len + buffer + read_len, samplingDivider),     # THIS LINE IS WRONG, FIX
             wave_index=0,
             ct_index=1,
            )
    

In [396]:
voltToDbm(0.34, chan["P2"].output.range())

0.9906716635319219

#### Experiment functions

In [397]:
def runfeedback():
    """Run feedback measurement where we only measure"""
    device.system.internaltrigger.enable(0)

    result_node = chan["measure"].spectroscopy.result.data.wave
    result_node.subscribe()
    
    chan["measure"].spectroscopy.result.enable(1)  # start logger
    chan["measure"].generator.enable_sequencer(single=True)
    device.system.internaltrigger.enable(1)
    
    time.sleep(0.2)  # delay for networking issues
        
    # wait for the measurement to complete
    waitForInternalTrigger()
    
    m_state = chan["measure"].generator.sequencer.status()
    if m_state != 4:
        warnings.warn(f"Sequencers in unknown state. Perhaps they are not synchronised? {bin(m_state)}")
        time.sleep(0.5)
    
    # wait for completion
    while chan["measure"].spectroscopy.result.enable() != 0:
        print(chan["measure"].spectroscopy.result.enable())
        chan["measure"].spectroscopy.result.enable.wait_for_state_change(0, timeout=10)
    
    # get results
    results = get_results(result_node, timeout=5)
    result_node.unsubscribe()
    
    # verify results
    acq = chan["measure"].spectroscopy.result.acquired()
    if len(results) > acq:
        print(chan["measure"].generator.ready())
        raise TimeoutError(f"Not all datapoints measured in the time provided. {acq} of {len(results)}.")

    return results
    

In [398]:
def runexperiment():
    device.system.internaltrigger.enable(0)

    result_node = chan["measure"].spectroscopy.result.data.wave
    result_node.subscribe()
    
    chan["measure"].spectroscopy.result.enable(1)  # start logger
    
    
    # start sequencers
    chan["measure"].generator.enable_sequencer(single=True)
    chan["J"].awg.enable_sequencer(single=True)  # dont want to repeat
    chan["P1"].awg.enable_sequencer(single=True)
    chan["P2"].awg.enable_sequencer(single=True)
    chan["ST"].awg.enable_sequencer(single=True)
    
    
    # start triggering sequence (which starts each sequencer)
    device.system.internaltrigger.enable(1)
    time.sleep(0.2)
    
    # wait for the measurement to complete
    waitForInternalTrigger(progress=True, leave=False)
    #device.system.internaltrigger. .wait_for_state_change(1.0, timeout=100)  # wait for completion


    # check sequencers have finished their sequences
    # Status of the Sequencer on the instrument.
    # - Bit 0: Sequencer is running;
    # - Bit 1: reserved;
    # - Bit 2: Sequencer is waiting for a trigger to arrive;
    # - Bit 3: Sequencer has detected an error;
    # - Bit 4: sequencer is waiting for synchronization with other channels
    m_state = chan["measure"].generator.sequencer.status()
    st_state = chan["P1"].awg.sequencer.status()
    if m_state != 4 and st_state != 4:
        warnings.warn(f"Sequencers in unknown state. Perhaps they are not synchronised? {bin(m_state)}, {bin(st_state)}")
        #time.sleep(0.5)
    
    
    # wait for completion
    while chan["measure"].spectroscopy.result.enable() != 0:
        chan["measure"].spectroscopy.result.enable.wait_for_state_change(0, timeout=100)

    
    # get results
    results = get_results(result_node, timeout=5)
    result_node.unsubscribe()
    
    
    # verify results
    acq = chan["measure"].spectroscopy.result.acquired()
    if len(results) > acq:
        print(chan["measure"].generator.ready())
        print([chan[c].awg.ready() for c in drive_chans])
        raise TimeoutError(f"Not all datapoints measured in the time provided. {acq} of {len(results)}.")
    
    #return np.mean(results.reshape((seq_averages, 2)), axis=0)
    return results




### Run loop

In [399]:
monty.runname

''

#### Sweep J and Detuning

In [400]:
monty.newrun("sequence", params)

# work out what we should be doing for feedback
print("Measuring initial feedback point")
with device.set_transaction():
    synchchannels([])
    chan["measure"].spectroscopy.result.length(1)
    chan["measure"].generator.load_sequencer_program(seqc_program_prior_read)

time.sleep(1)
chan["measure"].spectroscopy.result.length.wait_for_state_change(1, timeout=10)  # wait for changes to propagate

data = runfeedback()

with device.set_transaction():
    synchchannels(["P1", "P2", "J", "ST"])
    chan["measure"].generator.load_sequencer_program(readout_prog_code)
    chan["measure"].spectroscopy.result.length(2)
time.sleep(2)

result = np.zeros((NUM_J, NUM_DETUNING, 2), dtype=np.complex64)
st_result = np.zeros(result.shape, dtype=np.float64)

print("Sweeping P1, P2 and J")
for i, j in enumerate(tqdm(j_steps, desc="J")):
    #with LivePlot(p1_steps, xlabel="P1 voltage (dV)", ylabel="Amplitude (dB)") as lplot: 
    FB_STEPSIZE = steps[i]  # adjust the stepsize
    curr_st_amp = 0.0  # reset feedback between runs (as j is not varied)
    
    for k, (p1, p2) in enumerate(zip(tqdm(p1_steps, leave=False, desc="Detuning"), p2_steps)):
        movemeasurement(p1, p2, j)  # CT = 1
        st_result[i, k] = calculatefeedback(data)  # CT = 0
        uploadcts()
        data = runexperiment()
        result[i, k] = data
        #lplot.update(autodb(result[i, :, 1]))
    monty.snapshot({"data": result, "st_result": st_result})

monty.save({"data": result, "st_result": st_result})

Started new run sequence.175
Measuring initial feedback point


Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

Sweeping P1, P2 and J


J:   0%|          | 0/1 [00:00<?, ?it/s]

Detuning:   0%|          | 0/100 [00:00<?, ?it/s]

...

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

Internal trigger:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#monty.save({"data": result, "st_result": st_result})

### Process data

In [ ]:
# reload saved data

# result = monty.loadrun("sequence.95")["data"]

In [ ]:
result.shape

In [ ]:
ref = result[:, :, 0]
mes = result[:, :, 1]


ref_amp = 10*np.log10(np.abs(ref)**2/50*1000)
ref_phase = np.unwrap(np.angle(ref))  # add axis for unwrap?

mes_amp = 10*np.log10(np.abs(mes)**2/50*1000)
mes_phase = np.unwrap(np.angle(mes))  # add axis for unwrap?

In [ ]:
# plot ST feedback history

# convert "voltage" to dB
#st_result = voltToDbm(st_result, chan["ST"].output.range())

# plot the first sweep in J only
plt.plot(p1_steps, st_result[0])
plt.xlabel("P1 voltage (V)")
plt.ylabel("ST amplitude (a.u.)")
plt.title(monty.identifier + "." + monty.runname + ".st")
monty.savefig(plt, "st_history")


In [ ]:
# plot the FIRST reference measurement

fig, (ax0, ax1) = plt.subplots(nrows = 2, sharex = True)

ax0.plot(p1_steps, ref_amp[0], ".-", color="steelblue")
ax1.plot(p1_steps, ref_phase[0], ".-", color="orange")
ax0.set_ylabel("Amplitude")
ax1.set_ylabel("Phase")
ax0.set_title(monty.identifier + "." + monty.runname + ".reference.1st")
ax1.set_xlabel("P1 voltage")

ax0.grid()
ax1.grid()
fig.align_ylabels([ax0, ax1])
plt.tight_layout()

monty.savefig(plt, "reference_1st")

In [ ]:
# plot the FIRST measurement point

fig, (ax0, ax1) = plt.subplots(nrows = 2, sharex = True)

ax0.plot(p1_steps, mes_amp[0], ".-", color="steelblue")
ax1.plot(p1_steps, mes_phase[0], ".-", color="orange")

ax0.set_ylabel("Amplitude")
ax1.set_ylabel("Phase")
ax0.set_title(monty.identifier + "." + monty.runname + ".measurement.1st")
ax1.set_xlabel("P1 voltage")

ax0.grid()
ax1.grid()
fig.align_ylabels([ax0, ax1])
plt.tight_layout()

monty.savefig(plt, "measurement_1st")

In [ ]:
# plot the FIRST localised measurement (mes - ref)

a = mes_amp[0] - ref_amp[0]
p = mes_phase[0] - ref_phase[0]

fig, (ax0, ax1) = plt.subplots(nrows = 2, sharex = True)

ax0.plot(p1_steps, a, ".-", color="steelblue")
ax1.plot(p1_steps, p, ".-", color="orange")

ax0.set_ylabel("Amplitude")
ax1.set_ylabel("Phase")
ax0.set_title(monty.identifier + "." + monty.runname + ".adjusted")
ax1.set_xlabel("P1 voltage")
ax0.grid()
ax1.grid()

fig.align_ylabels([ax0, ax1])
plt.tight_layout()

monty.savefig(plt, "detuning minus ref 1st")

In [ ]:
plt.figure()
plt.pcolormesh(p1_steps, steps, mes_amp, shading="nearest")
plt.colorbar()
plt.ylabel("Feedback stepsize")
plt.xlabel("Detuning (V)")
plt.semilogy()
plt.title(monty.identifier + "." + monty.runname + ".mes_amp_stepsize")

monty.savefig(plt, "2d mes amp stepsize")

In [ ]:
# actually plot all the data

# MEASUREMENT ONLY

plt.figure()
plt.pcolormesh(p1_steps, j_steps, mes_amp, shading="nearest")
plt.colorbar()
plt.ylabel("J voltage (V)")
plt.xlabel("Detuning (V)")
plt.title(monty.identifier + "." + monty.runname + ".mes_amp")

if j_steps.shape[0] > 1:
    monty.savefig(plt, "2d mes amp")
#plt.show()



In [ ]:
plt.figure()
plt.pcolormesh(p1_steps, j_steps, mes_phase, shading="nearest")
plt.colorbar()
plt.ylabel("Averages")
plt.xlabel("Detuning (V)")
plt.title(monty.identifier + "." + monty.runname + ".mes_phase")
#monty.savefig(plt, "stability forward")
plt.show()

In [ ]:
getvoltages(mdac)

In [ ]:
#si.SLB(0.93)

In [ ]:
0.475/2+1.625